In [2]:
import polars as pl
import pandas as pd
import numpy as np
import multiprocessing

In [3]:
paramsList = [3,1,15,5]

In [4]:
openPdAverages=[]
for k in range(10,1205,5):
    openPdAverages.append(pd.read_csv("../Data/sp500/sma/2017/Open/SimpleMovingAverages"+str(k)+".csv"))
stocks=list(openPdAverages[0].columns)
closePdAverages=[]
for k in range(10,1205,5):
    closePdAverages.append(pd.read_csv("../Data/sp500/sma/2017/Close/SimpleMovingAverages"+str(k)+".csv"))

In [5]:
openPlAverages=[]
for k in range(10,1205,5):
    openPlAverages.append(pl.read_csv("../Data/sp500/sma/2017/Close/SimpleMovingAverages"+str(k)+".csv", infer_schema_length=None))
stocks=list(openPlAverages[0].columns)

In [ ]:
test=openPdAverages
total=0
momentumWorks=0
averagePercent=0
current=test[0]
for k in range(0, 239):
    for j in range(len(stocks)):
        if openPlAverages[k][paramsList[0],stocks[j]] > openPlAverages[k][paramsList[1],stocks[j]]:
            total+=1
            averagePercent+=((openPlAverages[k][paramsList[2],stocks[j]])/(openPlAverages[k][paramsList[3],stocks[j]]))
            if openPlAverages[k][paramsList[2],stocks[j]] > openPlAverages[k][paramsList[3],stocks[j]]:
                momentumWorks+=1

In [68]:
%%timeit -n250
current.loc[16] = (current.iloc[paramsList[0]] > current.iloc[paramsList[1]]).astype(int)
current.loc[17] = (current.iloc[paramsList[2]] > current.iloc[paramsList[3]]).astype(int)
current.loc[18] = ((current.iloc[paramsList[2]])/(current.iloc[paramsList[3]]))
total = sum(current.loc[16])
momentumWorks = sum(current.loc[17])
averagePercent = sum(current.loc[18])

614 µs ± 30.6 µs per loop (mean ± std. dev. of 7 runs, 250 loops each)


In [70]:
%%timeit -n25
total=0
momentumWorks=0
averagePercent=0
for j in range(len(stocks)):
    if openPlAverages[0][paramsList[0],stocks[j]] > openPlAverages[0][paramsList[1],stocks[j]]:
        total+=1
        averagePercent+=((openPlAverages[k][paramsList[2],stocks[j]])/(openPlAverages[k][paramsList[3],stocks[j]]))
        if openPlAverages[k][paramsList[2],stocks[j]] > openPlAverages[k][paramsList[3],stocks[j]]:
            momentumWorks+=1

8.08 ms ± 438 µs per loop (mean ± std. dev. of 7 runs, 25 loops each)


In [34]:
%%timeit -n1
#Pandas
import pandas as pd
import pickle
averagePercent=0
momentumWorks=0
total=0
test=openPdAverages
for k in range(0, 239):
    current=test[k]
    for j in range(len(stocks)):
        if current[stocks[j]][paramsList[0]] > current[stocks[j]][paramsList[1]]:
            total+=1
            averagePercent+=((current[stocks[j]][paramsList[2]])/(current[stocks[j]][paramsList[3]]))
            if current[stocks[j]][paramsList[2]] > current[stocks[j]][paramsList[3]]:
                momentumWorks+=1

1.39 s ± 380 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [91]:
%%timeit -n5
test=openPdAverages
averagePercent=0
momentumWorks=0
total=0
for k in range(0, 239):
    current=test[k]
    current.loc[16] = (current.iloc[paramsList[0]] > current.iloc[paramsList[1]]).astype(int)
    current.loc[17] = ((current.iloc[paramsList[2]] > current.iloc[paramsList[3]]) & (current.iloc[paramsList[0]] > current.iloc[paramsList[1]])).astype(int)
    current.loc[18] = (current.iloc[paramsList[2]])/(current.iloc[paramsList[3]])*((current.iloc[paramsList[0]] > current.iloc[paramsList[1]]).astype(int))
    total = sum(current.loc[16])
    momentumWorks = sum(current.loc[17])
    averagePercent = sum(current.loc[18])

211 ms ± 2.97 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [94]:
%%timeit -n5
openTest=openPdAverages
closeTest=closePdAverages
averagePercent=0
momentumWorks=0
total=0
for k in range(0, 239):
    openCurrent=openTest[k]
    closeCurrent=closeTest[k]
    total += sum((closeCurrent.iloc[paramsList[0]] > closeCurrent.iloc[paramsList[1]]).astype(int))
    momentumWorks += sum(((closeCurrent.iloc[paramsList[2]] > openCurrent.iloc[paramsList[3]]) & (current.iloc[paramsList[0]] > current.iloc[paramsList[1]])).astype(int))
    averagePercent += sum((closeCurrent.iloc[paramsList[2]])/(openCurrent.iloc[paramsList[3]])*((current.iloc[paramsList[0]] > current.iloc[paramsList[1]]).astype(int)))

181 ms ± 1.24 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)
